#### Embeddings for LLM

In [2]:
from importlib.metadata import version

import tiktoken
import torch

print("torch version: ", version("torch"))
print("tiktoken version: ", version("tiktoken"))

torch version:  2.8.0
tiktoken version:  0.11.0
